# Clustering - K-means, Hierarchical Clustering, DBSCAN 

In this lab we will perform unsupervised classification using clustering algorithms. This will give you an opportunity to explore different clustering methods and different setups for those methods in order to get an intuition as to how the process of performing unsupervised classifiation looks like.

## Dataset

First, we have our dataset. For this lab we will work with a few categories from the Reuters dataset. The code below will download and preprocess the dataset in order to allow us to spend more time on the actual techniques.

First, download the dataset to the current folder.

In [1]:
import numpy as np
import nltk
nltk.download('reuters', download_dir = './')

[nltk_data] Downloading package reuters to ./...


True

Secondly, go to the corpora folder located at the current path and unzip the reuters archive. Subsequently, run the rest of the processing code.

In [2]:
dataset_folder = './corpora/reuters'
with open(f'{dataset_folder}/cats.txt', 'r') as f:
    annotations = f.readlines()

selected_categories = ['sugar', 'livestock', 'jobs', 'ship']
category_to_index = dict(zip(selected_categories, range(len(selected_categories))))

train_texts, train_labels, test_texts, test_labels = [], [], [], []
for ann in annotations:
    ann = ann.rstrip().split()
    
    if not any([category in ann for category in selected_categories]):
        continue
    
    document_text = open(f'{dataset_folder}/{ann[0]}', 'r').read()
    label = category_to_index[
        [category for category in selected_categories if category in ann[1:]][0]
    ]
    
    if 'train' in ann[0]:
        train_texts.append(document_text)
        train_labels.append(label)
    else:
        test_texts.append(document_text)
        test_labels.append(label)
    
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

After this preprocessing we end up with 4 arrays.
- train_texts - which is a list of all the texts in the train dataset
- train_labels - which is a numpy array with the labels of train_texts, corresponding to the chosen categories
- test_texts - which is a list of all the texts in the test dataset
- test_labels - which is a numpy array with the labels of test_texts, corresponding to the chosen categories

## Preprocessing

For this exercise we will use a simple TF-IDF vectorizer from the sklearn library.

Your first task is to compute the train_data and test_data variables, which should be the results of applying the TfidfVectorizer from the sklearn library on our dataset. Use a maximum of 500 features. Fit the vectorizer on the training texts and use it to transform both training and test documents.

In [235]:
with open('./corpora/reuters/stopwords', 'r+') as f:
  stopwords = [line.removesuffix('\n') for line in f.readlines()]

In [236]:
import sklearn as sk
import sklearn.feature_extraction.text as text

tf_idf = text.TfidfVectorizer(max_features=1000, stop_words=stopwords)
tf_idf.fit(train_texts)

train_data = tf_idf.transform(train_texts)
test_data = tf_idf.transform(test_texts)

## Sanity check

To ensure that everything is okay with our data and preprocessing, and, in order to have a baseline as a reference, fit an SVC on the training data and evaluate it on the test split. Report your results.

In [237]:
import sklearn.svm as svm
import sklearn.metrics as metrics

model = svm.SVC(kernel='linear')
model.fit(train_data, train_labels)
y_hat = model.predict(test_data)

test_accuracy = metrics.accuracy_score(test_labels, y_hat)
print(test_accuracy)

0.9578313253012049


## K-Means

The first clustering algorithm we will investigate is the one we've already seen: K-Means.

As a basline, fit the a K-Means model, without any change in parameters on the dataset.

In [238]:
from sklearn.cluster import KMeans

model_cluster = KMeans(n_clusters=4)
model_cluster.fit(train_data)

/home/invokariman/.cache/pypoetry/virtualenvs/lab6-vTFSZr9J-py3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=4)

### Evaluation

Since we work in an unsupervised classification scenario, we will evaluate or model with respect to the class labels that we have. In order to do that, we have to match each cluster to a class (since clusters are not ordered in any particular order). For that, we will compute the confusion matrix (m\[i\]\[j\] = number of samples from class i assigned to cluster j) on the training set and use it do determine the best matching.

In order to perform the matching we will use the linear_sum_assignment method implemented in the scipy.optimize package (https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.linear_sum_assignment.html). This method requires a cost function in order to compute the best matching. For that, we will feed the inverse of the confusion matrix, that is 1.0/confusion_matrix.

Once you compute the best matching, translate the cluster labels into class labels and evaluate the accuracy of the model on the training set as well as the test set.

### Exercises:

Make changes such as the following and take note on how those influence the performance of the model.

1. Consider the fact that the classes are unbalanced. In order to account for that in the cost matrix, divide each row of the confusion matrix by its sum before passing it to the linear_sum_assignment procedure.
2. Insted of passing the inverse of the confusion_matrix as a cost, pass the negative of the confusion matrix
3. Try the k-means++ init for the clustering algorithm
4. Try different values for the n_init parameter
5. Try using PCA or t-sne on your data
6. Try using different amount of features when computing the TF-IDF representations
7. Try using the stopwords provided with the dataset during the TF-IDF vectorization (nltk_data/corpora/reuters/stopwords)
9. Try using a different vectorization procedure
10. Try normalizing your data

In [350]:
from scipy.optimize import linear_sum_assignment

# research on the problem at hand
model_cluster = KMeans(n_clusters=4, n_init=100, init='k-means++', max_iter=3_000)
model_cluster.fit(train_data)

# evaluate the unsupervised method
y_train = model_cluster.predict(train_data)
y_test = model_cluster.predict(test_data)

# compute conf matrix assignments
conf_train = metrics.confusion_matrix(train_labels, y_train)

# divide by sum over columns
row_sums = np.tile(conf_train.sum(axis=1), (1, 4)).reshape(4, 4).T

# compute the cost matrix (inverse of conf matrix)
cost_train = -1.0 * (conf_train / row_sums)

# # optimize by matching pairs row-col wise using hungarian alg.
r_train, c_train = linear_sum_assignment(cost_train)

# swap labels
y_train_ = np.zeros_like(y_train)
y_test_ = np.zeros_like(y_test)

for r, c in zip(r_train, c_train):
  y_train_[np.where(y_train == r)] = c
  y_test_[np.where(y_test == r)] = c

# compute conf matrix assignments
conf_train_ = metrics.confusion_matrix(train_labels, y_train_)
conf_test_ = metrics.confusion_matrix(test_labels, y_test_)

# compute accuracy
train_acc = metrics.accuracy_score(train_labels, y_train_)
test_acc = metrics.accuracy_score(test_labels, y_test_)

# show accuracy
print('train acc', train_acc)
print('test acc', test_acc)

train acc 0.5253456221198156
test acc 0.6506024096385542


### Consider the realistic scenario

Consider an actual project where you are given a set of documents collected from the web, that do not have any human annotations. You are tasked to group those documents semantically.

Choose one of the models developed in the previous phase.

1. Look through the cluster texts in order to get a sense of what each cluster represents.
2. For each cluster extract the most important/characteristic words and see what they point to
    - you can do this by evaluating documents that are closer to the center of the cluster
    - you can do this by running statistics on the entire cluster
    - you can use different measures for the importance of a word (number of distinct documents within which the word appears, tfidf for the documents closer to the center, etc)
    
Use this exercise in order to get a sense of the data and to get a sense of how the clustering algorithm grouped them and what those groups represent.

In [ ]:
# interpretating results

## Hierarchical Clustering

We will continue the investigation of our problem using a different clustering procedure, namely agglomerative clustering. For that we will use the implemenation provided by the scipy library (https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html).

### Exercises

Repeat evaluating the different setups presented in the previous set of exercises in order to investigate the current clustering method. In addition:

1. Evaluate different linking methods
2. Evaluate different metrics (such as cosine)

And see how they impact the performance and learning process.

In [ ]:
# research on the problem at hand

In [ ]:
# interpreting results

## DBSCAN

Finally, we will make use of the DBSCAN algorithm in order to cluster the data (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html).

### Exercises

Investigate the DBSCAN model. In addition to the previous setups:

1. Evaluate the distribution of distances between samples in the training data and use them as a reference point when deciding the parameters
2. Perform a grid search on the parameters
3. Use a different association rule for evaluation, which allows multiple clusters to pe assigned to a single class, for instance, based on the confusion matrix, assign each cluster to the class with which it has the most samples in common

In [ ]:
# research on the problem at hand

In [ ]:
# interpreting results